# Autoscaling and concurrency in Ray Serve example
This notebook explains
1. How to expose a Python function as a Ray Serve API.
2. What parameters are available to tweak the performance of the Ray Serve application
3. How the Ray serve application scales when invoked concurrently

In [15]:
RAY_SERVE_URL = "http://ray-cluster-serve.mlnow.frenoid.com:30080"
ROUTE_PREFIX = "/mprimes"

In [16]:
import ray
import os

os.environ["RAY_ADDRESS"] = "ray://192.168.3.201:30101"

ray.init()

2025-03-24 11:13:51,956	INFO worker.py:1514 -- Using address ray://192.168.3.201:30101 set in the environment variable RAY_ADDRESS
2025-03-24 11:13:51,957	INFO client_builder.py:244 -- Passing the following kwargs to ray.init() on the server: log_to_driver


RuntimeError: Ray Client is already connected. Maybe you called ray.init("ray://<address>") twice by accident?

## Mersenne Primes calculation function

In [3]:
@ray.remote
def generate_mersenne_primes(start, end):
    m_primes = []
    def lucas_lehmer(p):
        s = 4
        m = 2 ** p - 1
        for _ in range(p - 2):
            s = ((s * s) - 2) % m
        return s == 0

    def is_prime(number):
        """
        the efficiency of this doesn't matter much as we're
        only using it to test the primeness of the exponents
        not the mersenne primes themselves
        """

        if number % 2 == 0:
            return number == 2

        i = 3
        while i * i <= number:
            if number % i == 0:
                return False
            i += 2

        return True
    
    for i in range(start, end, 2):  # generate up to M20, found in 1961
        if is_prime(i) and lucas_lehmer(i):
            m_primes.append(2 ** i - 1)
    
    return m_primes

In [4]:
ref = generate_mersenne_primes.remote(1, 500)
ray.get(ref)

[7,
 31,
 127,
 8191,
 131071,
 524287,
 2147483647,
 2305843009213693951,
 618970019642690137449562111,
 162259276829213363391578010288127,
 170141183460469231731687303715884105727]

### Calling the function concurrently
You can do so using ray.remote and the futures package

In [5]:
import concurrent.futures

CONCURRENCY = 3
TOTAL_CALLS = 30

START = 1
STOP = 1000

def call_mprime(start: int, stop: int) -> list[int]:
    ref = generate_mersenne_primes.remote(start, stop)
    return ray.get(ref)

pool = concurrent.futures.ThreadPoolExecutor(max_workers=CONCURRENCY)

futures = []
for i in range(TOTAL_CALLS):
    futures.append(pool.submit(call_mprime, START, STOP))
    
responses, response_count = 0, 0
for f in futures:
    res = f.result()
    response_count += 1
    print(f"responses: {response_count}")
    #print(res)

responses: 1
responses: 2
responses: 3
responses: 4
responses: 5
responses: 6
responses: 7
responses: 8
responses: 9
responses: 10
responses: 11
responses: 12
responses: 13
responses: 14
responses: 15
responses: 16
responses: 17
responses: 18
responses: 19
responses: 20
responses: 21
responses: 22
responses: 23
responses: 24
responses: 25
responses: 26
responses: 27
responses: 28
responses: 29
responses: 30


## Deploying Mersenne Primes function Ray Serve application

### First startup  Ray Serve

In [7]:
! pip3 install --quiet ray[serve]==2.41.0

In [8]:
from ray import serve

serve.start(http_options={"location": "EveryNode",
                          "host": "0.0.0.0",
                          "port": 8000,
                          "request_timeout_s": 180.0})

(ProxyActor pid=2889) INFO 2025-03-23 11:48:48,571 proxy 10.244.1.204 -- Proxy starting on node c20f9bed881279fd0faa8c54869a977dd0d3a842e0169fd80ea68ffc (HTTP port: 8000).
INFO 2025-03-23 18:48:48,633 serve 2523374 -- Started Serve in namespace "serve".
(ProxyActor pid=2889) INFO 2025-03-23 11:48:48,626 proxy 10.244.1.204 -- Got updated endpoints: {}.


### Declare and deploy the Ray Serve application

In [39]:
from ray import serve
from ray.serve.handle import DeploymentHandle
from starlette.requests import Request

@serve.deployment(
    ray_actor_options={"num_cpus": 1.0, "num_gpus": 0},
    autoscaling_config={
        "target_num_ongoing_requests_per_replica": 2,
        "min_replicas": 1,
        "initial_replicas": 1,
        "max_replicas": 5
    },
)
class MersennePrimes:
    def __init__(self):
        print("MersennePrimes is ready to serve")
    
    def generate_mersenne_primes(self, start, end) -> list[int]:    
        def lucas_lehmer(p):
            s = 4
            m = 2 ** p - 1
            for _ in range(p - 2):
                s = ((s * s) - 2) % m
            return s == 0

        def is_prime(number):
            """
            the efficiency of this doesn't matter much as we're
            only using it to test the primeness of the exponents
            not the mersenne primes themselves
            """

            if number % 2 == 0:
                return number == 2

            i = 3
            while i * i <= number:
                if number % i == 0:
                    return False
                i += 2

            return True
        
        m_primes = []
        for i in range(start, end, 2):  # generate up to M20, found in 1961
            if is_prime(i) and lucas_lehmer(i):
                m_primes.append(2 ** i - 1)

        return m_primes

    async def __call__(self, request: Request) -> list[str]:
        start = int((await request.json())["start"])
        stop = int((await request.json())["stop"])
        print(f"MersennePrimes called with {start} {stop}")
        m_primes = self.generate_mersenne_primes(start, stop)
        
        return m_primes

In [40]:
app = MersennePrimes.bind()
handle: DeploymentHandle = serve.run(app, name="MersennePrimes", route_prefix=ROUTE_PREFIX)

INFO 2025-03-24 11:41:34,239 serve 2523374 -- Connecting to existing Serve app in namespace "serve". New http options will not be applied.
WARNING 2025-03-24 11:41:34,240 serve 2523374 -- The new client HTTP config differs from the existing one in the following fields: ['host', 'request_timeout_s']. The new HTTP config is ignored.
(ServeController pid=2823) INFO 2025-03-24 04:41:34,267 controller 2823 -- Deploying new version of Deployment(name='MersennePrimes', app='MersennePrimes') (initial target replicas: 1).
(ServeController pid=2823) INFO 2025-03-24 04:41:34,370 controller 2823 -- Stopping 1 replicas of Deployment(name='MersennePrimes', app='MersennePrimes') with outdated versions.
(ServeController pid=2823) INFO 2025-03-24 04:41:34,370 controller 2823 -- Adding 1 replica to Deployment(name='MersennePrimes', app='MersennePrimes').


(ServeReplica:MersennePrimes:MersennePrimes pid=200871, ip=10.244.1.193) MersennePrimes is ready to serve


(ProxyActor pid=200934, ip=10.244.1.193) INFO 2025-03-24 04:41:35,587 proxy 10.244.1.193 -- Proxy starting on node ec025697127fbd672fccfe1fcfb5634113b006fc19220d743db28da8 (HTTP port: 8000).
(ProxyActor pid=200934, ip=10.244.1.193) INFO 2025-03-24 04:41:35,634 proxy 10.244.1.193 -- Got updated endpoints: {Deployment(name='MersennePrimes', app='MersennePrimes'): EndpointInfo(route='/mprimes', app_is_cross_language=False)}.
(ServeController pid=2823) INFO 2025-03-24 04:41:36,429 controller 2823 -- Replica(id='kcvfskrl', deployment='MersennePrimes', app='MersennePrimes') is stopped.
INFO 2025-03-24 11:41:37,259 serve 2523374 -- Application 'MersennePrimes' is ready at http://0.0.0.0:8000/mprimes.
INFO 2025-03-24 11:41:37,260 serve 2523374 -- Deployed app 'MersennePrimes' successfully.


### Declare and deploy the Ray Serve application

In [41]:
app_url = f"{RAY_SERVE_URL}{ROUTE_PREFIX}"

print(app_url)

http://ray-cluster-serve.mlnow.frenoid.com:30080/mprimes


In [42]:
import requests

start = 3
stop = 1000

res = requests.get(app_url,
                   json={"start":start, "stop": stop})
res

<Response [200]>

(ServeReplica:MersennePrimes:MersennePrimes pid=200871, ip=10.244.1.193) MersennePrimes called with 3 1000


(ServeReplica:MersennePrimes:MersennePrimes pid=200871, ip=10.244.1.193) INFO 2025-03-24 04:41:41,006 MersennePrimes_MersennePrimes qnytibs0 6c88025696ef45175c781a85bb63b64a -- GET /mprimes 200 65.5ms


In [43]:
res.json()

[7,
 31,
 127,
 8191,
 131071,
 524287,
 2147483647,
 2305843009213693951,
 618970019642690137449562111,
 162259276829213363391578010288127,
 170141183460469231731687303715884105727,
 6864797660130609714981900799081393217269435300143305409394463459185543183397656052122559640661454554977296311391480858037121987999716643812574028291115057151,
 531137992816767098689588206552468627329593117727031923199444138200403559860852242739162502265229285668889329486246501015346579337652707239409519978766587351943831270835393219031728127]

### Call the Ray Serve application concurrently

Observe the replica scaling

In [46]:
import concurrent.futures
import random
from time import sleep

CONCURRENCY = 5
TOTAL_CALLS_PER_ITERATION = 15
ITERATIONS = 200_000_000
PROBABILTY_OF_BAD_REQUEST=0.05

def invoke_mprime_application(start: int, stop: int) -> list[int]:
    res = requests.get(app_url,
                   json={"start":start, "stop": stop})
    
    return res.json()

def bad_request():
    try:
      print("BAD REQUEST")
      res = request.get(app_url, json={"start": "start", "stop": "stop"})
      res.json()
    except Exception:
        pass
    
    return

In [47]:
bad_request()

BAD REQUEST


In [ ]:
for i in range(ITERATIONS):
    print(f"Begin iteration {i}")
    sleep_seconds = random.randint(1,60)
    start = random.randint(3, 100)
    stop = random.randint(240,4000)
    
    print(f"Params: sleep_Seconds {sleep_seconds}, start {start}, stop {stop}")
    pool = concurrent.futures.ThreadPoolExecutor(max_workers=CONCURRENCY)
    
    futures = []
    for _ in range(TOTAL_CALLS_PER_ITERATION):
        futures.append(pool.submit(invoke_mprime_application, start, stop))
        
    response_count = 0
    for f in futures:
        res = f.result() # Blocks until result is returned
        if random.random() < PROBABILTY_OF_BAD_REQUEST:
            bad_request()
        response_count += 1
        
    print(f"Responses received: {response_count}")
    print(f"Iteration {i} complete")
    sleep(sleep_seconds)

Begin iteration 0
Params: sleep_Seconds 2, start 52, stop 2494
BAD REQUEST
BAD REQUEST
BAD REQUEST
Responses received: 15
Iteration 0 complete
(ServeReplica:MersennePrimes:MersennePrimes pid=200871, ip=10.244.1.193) MersennePrimes called with 52 2494
(ServeReplica:MersennePrimes:MersennePrimes pid=200871, ip=10.244.1.193) MersennePrimes called with 52 2494
(ServeReplica:MersennePrimes:MersennePrimes pid=200871, ip=10.244.1.193) MersennePrimes called with 52 2494
(ServeReplica:MersennePrimes:MersennePrimes pid=200871, ip=10.244.1.193) MersennePrimes called with 52 2494
(ServeReplica:MersennePrimes:MersennePrimes pid=200871, ip=10.244.1.193) MersennePrimes called with 52 2494
(ServeReplica:MersennePrimes:MersennePrimes pid=200871, ip=10.244.1.193) MersennePrimes called with 52 2494
(ServeReplica:MersennePrimes:MersennePrimes pid=200871, ip=10.244.1.193) MersennePrimes called with 52 2494
(ServeReplica:MersennePrimes:MersennePrimes pid=200871, ip=10.244.1.193) MersennePrimes called with 5

(ServeReplica:MersennePrimes:MersennePrimes pid=200871, ip=10.244.1.193) INFO 2025-03-24 04:43:22,669 MersennePrimes_MersennePrimes qnytibs0 7f4a9a76344ae56dd9794686da78449a -- GET /mprimes 200 3.4ms
(ServeReplica:MersennePrimes:MersennePrimes pid=200871, ip=10.244.1.193) INFO 2025-03-24 04:43:22,671 MersennePrimes_MersennePrimes qnytibs0 83b10716b90839c2006d67ece53da3c6 -- GET /mprimes 200 4.4ms
(ServeReplica:MersennePrimes:MersennePrimes pid=200871, ip=10.244.1.193) INFO 2025-03-24 04:43:22,672 MersennePrimes_MersennePrimes qnytibs0 4715149b6a1184b56657bc8de0e0a604 -- GET /mprimes 200 4.3ms
(ServeReplica:MersennePrimes:MersennePrimes pid=200871, ip=10.244.1.193) INFO 2025-03-24 04:43:22,672 MersennePrimes_MersennePrimes qnytibs0 483de2c311c7a07fe032b72ff79c17d4 -- GET /mprimes 200 3.9ms
(ServeReplica:MersennePrimes:MersennePrimes pid=200871, ip=10.244.1.193) INFO 2025-03-24 04:43:22,672 MersennePrimes_MersennePrimes qnytibs0 b27a3269227a2797d3362c214d85f7e0 -- GET /mprimes 200 4.0ms


Begin iteration 1
Params: sleep_Seconds 57, start 35, stop 1670
(ServeReplica:MersennePrimes:MersennePrimes pid=200871, ip=10.244.1.193) MersennePrimes called with 35 1670
(ServeReplica:MersennePrimes:MersennePrimes pid=200871, ip=10.244.1.193) MersennePrimes called with 35 1670


(ServeReplica:MersennePrimes:MersennePrimes pid=200871, ip=10.244.1.193) INFO 2025-03-24 04:43:25,104 MersennePrimes_MersennePrimes qnytibs0 bc35171281f5dc70b774a99eaa9b8212 -- GET /mprimes 200 404.5ms


(ServeReplica:MersennePrimes:MersennePrimes pid=200871, ip=10.244.1.193) MersennePrimes called with 35 1670


...

(ServeReplica:MersennePrimes:MersennePrimes pid=264037) MersennePrimes called with 97 3728


(ServeReplica:MersennePrimes:MersennePrimes pid=264036) INFO 2025-03-24 04:49:49,186 MersennePrimes_MersennePrimes vqf068fk a3cbc6d53612f5fa78f8653ea51f4a33 -- GET /mprimes 200 8250.2ms


(ServeReplica:MersennePrimes:MersennePrimes pid=264036) MersennePrimes called with 97 3728


(ServeReplica:MersennePrimes:MersennePrimes pid=264037) INFO 2025-03-24 04:49:57,109 MersennePrimes_MersennePrimes gdlyhmnd d3873e412c5c04d6e671922c0611ab3f -- GET /mprimes 200 16172.8ms


(ServeReplica:MersennePrimes:MersennePrimes pid=264037) MersennePrimes called with 97 3728
(ServeReplica:MersennePrimes:MersennePrimes pid=264036) MersennePrimes called with 97 3728


## Cleanup

In [39]:
from ray import serve

serve.delete("MersennePrimes")

INFO 2025-03-01 17:55:04,600 serve 2116265 -- Deleting app ['MersennePrimes']


In [40]:
ray.shutdown()